# Download SmartSeq2 Expression Matrix as an Input to Scanpy

In this notebook, we demonstrate how to obtain a Smartseq2 expression matrix programmatically using the API for the Human Cell Atlas's Data Store (DSS).

Start by importing the HCA's Python API and creating a DSS API client:

In [1]:
import hca.dss
client = hca.dss.DSSClient()

In order to find an expresison matrix to analyze with Scanpy, we need to search for bundles containing a `*.results` file. This file will contain the information needed to assemble the expression matrix.

The DSS uses ElasticSearch to index all of the data, so we must assemble an ElasticSearch query. We want to set two conditions with our query: first, we need the bundle to contain a file named `*.results`; and second, we are interested in more recent expression matrices, so we exclude results older than 7/12/2018. The query looks like this:

In [2]:
query = {
    "query": {
        "bool": {
            "must": [
                {
                    "wildcard": {
                        "manifest.files.name": {
                            # We need a *.results file...
                            "value": "*.results"
                        }
                    }
                },
                {
                    "range": {
                        "manifest.version": {
                            # ...and preferably not too old, either.
                            "gte": "2018-07-12T100000.000000Z"
                        }
                    }
                }
            ]
        }
    }
}

While this query is a bit hard to decipher, we can think about it this way:

* the **query** looks for bundles matching a **boolean** condition: that the two checks in the query **must** be true. 

* The **wildcard** check looks for the `manifest.files.name` field, which contains the bundle manifest, and returns true if the name of a file listed in the bundle manifest ends with `.results`.

* The **range** check returns true if the bundle's `manifest.version` has a value greater than or equal to 7/12/2018. (It is important to note that the DSS uses timestamps for version numbers, which makes it possible to implement the date restriction.)

In short, __this query will find bundles that contain a `.results` file and are newer than July 12, 2018.__

Next, we execute the search with `post_search()`:

In [1]:
# Use output_format='raw' to inculde metadata with search results
bundles = client.post_search(es_query=query, replica='aws', output_format='raw')

NameError: name 'client' is not defined

The metadata for search results are stored in a list under the `results` key. From there we can access information about the results, including the files list. We can use that to get the name of the `*.results` file we are after, and then download that file from the DSS:

In [4]:
the_first_bundle = bundles['results'][0]
bundle_files = the_first_bundle['metadata']['manifest']['files']

# Print part of a recent analysis bundle with a results file
for f in bundle_files:
    if f['name'].endswith('.results'):
        results_file_uuid, results_file_version = f['uuid'], f['version']
print(results_file_uuid, results_file_version)

7806bb3a-d944-4d6d-bc4d-a9765f97e6ff 2019-02-11T124829.023428Z


The UUID of the file uniquely identifies this file in the entire DSS, so we can pass it to the `get_file()` API call to download the file from the data store. We use the API to get the data and write it to a file:

In [39]:
results = client.get_file(replica='aws', uuid=results_file_uuid, version=results_file_version)
open('matrix.results', 'w').write(results.decode("utf-8"))

18888302

Here are the first few lines of the `matrix.results` file we have now obtained:

In [40]:
print(open('matrix.results', 'r').read()[:852])

transcript_id	gene_id	length	effective_length	expected_count	TPM	FPKM	IsoPct	posterior_mean_count	posterior_standard_deviation_of_count	pme_TPM	pme_FPKM	IsoPct_from_pme_TPM
ENST00000373020.8	ENSG00000000003.14	2206	1990.09	35.20	92.29	87.70	95.76	34.74	1.37	40.87	79.04	90.13
ENST00000494424.1	ENSG00000000003.14	820	604.09	0.00	0.00	0.00	0.00	0.11	0.41	0.81	1.56	1.78
ENST00000496771.5	ENSG00000000003.14	1025	809.09	0.00	0.00	0.00	0.00	0.17	0.60	0.77	1.49	1.70
ENST00000612152.4	ENSG00000000003.14	3796	3580.09	2.80	4.09	3.88	4.24	2.79	1.05	1.88	3.65	4.16
ENST00000614008.4	ENSG00000000003.14	900	684.09	0.00	0.00	0.00	0.00	0.20	0.68	1.01	1.96	2.24
ENST00000373031.4	ENSG00000000005.5	1339	1123.09	0.00	0.00	0.00	0.00	0.00	0.00	0.21	0.40	22.50
ENST00000485971.1	ENSG00000000005.5	542	326.09	0.00	0.00	0.00	0.00	0.00	0.00	0.72	1.38	77.50
ENST000003715


Suppose we are only interested in some of these fields, for example the `gene_id`, `TPM`, and `FPKM` columns. We can use Python's `csv` module to read the data from the results file, filter it, and write it to a new file.

We can open a reader and a writer at the same time, and trim columns on each row as they are read in from `matrix.results`.

In [59]:
import csv

# Take the data we want out of the results file and store it into a tsv file
with open('matrix.results', 'r') as infile, open('matrix.tsv', 'w', newline='') as outfile:
    reader = csv.DictReader(infile, delimiter='\t')
    writer = csv.DictWriter(outfile, fieldnames=['gene_id', 'TPM', 'FPKM'], delimiter='\t')
    writer.writeheader()
    for row in reader:
        writer.writerow({'gene_id': row['gene_id'], 'TPM': row['TPM'], 'FPKM': row['FPKM']})

Our new file, `matrix.tsv`, looks something like this:

In [60]:
print("".join(open('matrix.tsv', 'r').readlines()[:8]))

gene_id	TPM	FPKM
ENSG00000000003.14	92.29	87.70
ENSG00000000003.14	0.00	0.00
ENSG00000000003.14	0.00	0.00
ENSG00000000003.14	4.09	3.88
ENSG00000000003.14	0.00	0.00
ENSG00000000005.5	0.00	0.00
ENSG00000000005.5	0.00	0.00



Now that we have a file containing the expression matrix, we can transpose it and read it into scanpy.

In [89]:
import scanpy as sc

adata = sc.read_csv(filename='matrix.tsv', delimiter='\t').transpose()
print(adata)

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


AnnData object with n_obs × n_vars = 2 × 200468 


We can use the Scanpy AnnData object to get information, such as particlar observations or the names of all variables:

In [93]:
print(adata.obs_vector(0))
print(list(adata.var_names)[15])

[92.29 87.7 ]
ENSG00000000457.13


We can also access the raw data using the `adata.X` attribute:

In [97]:
print("Var 1:")
for i in range(0, 153):
    print('{:<6}'.format('{:.1f}'.format(adata.X[0][i])), end='' if (i + 1) % 17 != 0 else '\n' )

Var 1:
92.3  0.0   0.0   4.1   0.0   0.0   0.0   0.0   0.0   194.4 0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0.0   0.0   43.9  102.7 0.0   3.2   0.0   35.6  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0.0   0.0   0.0   0.0   0.0   0.0   78.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0.0   0.0   0.0   0.0   0.0   20.1  69.3  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0.0   18.4  20.9  53.4  0.0   0.0   0.0   0.0   135.5 0.0   0.0   145.7 0.0   0.0   0.0   0.0   0.0   
0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   7.4   0.0   0.0   0.0   0.0   0.0   39.8  


And just to make it easier to see the relevant data in our matrix...

In [98]:
for i in range(len(adata.X[0])-1):
    if adata.X[0][i] != 0:
        print(adata.X[0][i], end=' ')

92.29 4.09 194.38 43.88 102.69 3.2 35.63 78.03 20.08 69.27 18.39 20.91 53.39 135.53 145.66 7.41 39.77 19.23 31.46 7.1 11.71 4.61 7.1 11.71 4.61 9.85 15.63 7.38 27.65 4.76 23.93 72.49 71.73 58.74 4.35 3.65 4.34 1.21 19.4 10.25 51.49 11.59 12.52 21.91 45.37 4.91 72.06 1.46 151.79 2.21 3.42 31.45 5.08 103.33 182.48 30.37 4.54 24.49 75.91 114.08 55.16 6.34 90.08 70.35 160.0 26.53 1.91 86.84 69.46 28.9 69.67 28.11 146.77 25.55 28.18 184.71 34.44 8.1 21.41 53.23 32.31 24.58 93.01 15.84 23.25 1.35 57.51 3.35 35.72 586.26 84.83 4.08 15.32 120.05 261.63 90.38 23.64 15.91 69.8 14.37 2.43 6.16 407.2 61.27 49.31 56.35 75.32 43.71 85.16 171.08 214.87 35.89 33.81 2.69 7.31 119.86 320.65 35.31 5.81 8.64 8.19 2.2 388.11 557.34 10.83 47.66 10.89 28.15 62.64 47.05 30.2 53.39 4.57 58.9 3.11 17.3 5.05 52.39 0.83 11.48 5.96 3.8 23.36 2.6 13.5 40.52 8.08 20.5 48.7 17.72 18.14 126.14 169.02 2.19 138.01 13.06 50.34 7.9 26.42 2.47 40.82 3.75 5.13 14.58 2.43 11.38 71.29 65.43 3.31 23.67 29.94 88.41 41.04 7.76 1

 7.68 5.1 27.16 13.21 94.48 15.57 16.57 2.12 55.02 16.2 149.15 63.62 3.22 0.48 2.38 6.45 3.76 40.8 1.91 5.37 144.87 8.07 149.38 107.52 8.14 0.01 8.14 4.83 13.49 84.7 76.52 58.21 47.78 76.05 12.94 8.44 19.43 15.62 55.36 21.47 77.85 31.11 4.21 15.38 44.39 83.46 67.01 55.91 43.96 40.41 28.05 41.74 83.17 11.03 4.22 2.7 3.18 19.73 7.12 11.0 88.72 7.3 19.25 103.41 18.5 11.58 11.77 88.92 41.28 5.14 42.48 29.73 63.31 12.78 12.02 6.26 7.2 17.99 57.57 29.09 312.98 131.82 30.51 1.46 109.03 63.51 61.04 39.91 15.18 29.31 785.14 2065.94 199.96 13.77 27.13 29.2 51.82 11.36 175.96 13.83 37.2 55.16 163.24 8.27 8.6 30.53 2.47 78.64 5.75 5.84 13.45 39.07 9.49 88.81 192.43 28.14 139.98 51.52 24.65 18.22 138.1 30.4 10.85 489.43 380.84 1024.23 186.38 24.74 5.69 95.42 34.13 100.42 128.96 19.57 25.72 7.84 69.47 50.77 39.11 156.64 7.07 5.73 2.99 49.32 3.76 21.61 49.85 0.84 33.15 1.6 141.18 43.5 50.18 32.48 7.08 7.37 15.27 33.49 263.5 352.0 21.59 27.42 29.49 0.95 13.76 67.51 120.96 16.5 93.27 215.8 229.64 30.43

13.04 38.88 31.3 5.69 16.84 9.44 17.21 5.69 530.81 54.88 379.27 145.78 5.23 2.19 7.01 40.17 20.7 27.23 21.85 17.42 14.05 5.33 15.91 20.15 68.29 138.88 65.16 43.11 4.63 63.28 21.98 71.26 24.39 33.72 53.89 102.91 50.82 94.49 14.98 8.03 57.63 66.85 61.92 9.18 66.48 2.27 17.03 9.83 574.34 3031.25 4.41 379.57 17283.83 7366.97 228.14 42.06 101.13 2.73 26.94 16.45 1229.38 604.41 350.78 31.32 1.47 9.16 31.22 37.9 23.28 22.32 107.74 108.97 200.42 53.23 101.17 10.21 27.75 11.16 25.5 16.17 15.51 33.44 3.26 1.63 22.12 228.99 6.39 8.78 48.3 21.9 6.38 69.06 29.46 13.28 6.32 166.93 13.63 57.48 24.28 16.3 98.71 60.9 4983.35 5167.94 33.69 2.36 36.77 21.49 60.15 31.15 63.53 11.65 55.83 0.6 9.64 103.57 1.76 45.42 12.61 18.46 12.95 0.71 27.33 22.98 21.36 26.11 4.26 183.09 188.44 38.04 109.44 2.03 2.72 10.39 90.29 2.46 6.01 34.77 147.24 53.98 14.53 14.14 90.22 74.95 9.14 11.62 263.42 9.15 3.91 4.98 6.94 20.79 6.7 105.64 16.97 46.62 28.52 21.45 4.7 0.99 10.57 7.94 102.03 6.28 102.56 2.23 25.51 24.45 3.63 18

 15.43 57.98 14.93 40.21 23.72 240.72 23.89 131.38 2.18 42.06 21.52 28.18 124.86 94.22 24.73 31.46 1.37 2.67 1265.08 19.16 41.49 44.64 1.28 10.62 0.99 2.17 9.13 8.31 86.7 449.43 13.21 16.43 22.39 4.63 144.19 57.32 50.08 26.97 15.62 20.68 18.16 29.39 1080.78 1866.15 389.93 15986.27 2459.77 8.23 16.25 2.29 12.71 32.3 44.05 22.0 52.08 7.62 33.73 101.34 247.53 69.97 20.52 38.07 34.91 58.02 7.62 33.37 151.31 2.51 14.06 67.03 31.15 29.8 47.81 9.58 19.03 23.85 10.54 4.56 2.46 54.21 13.55 11.22 35.67 28.14 233.41 1.9 29.91 83.73 143.58 127.87 54.76 1.9 31.41 16.66 77.84 17.96 181.14 38.72 27.45 34.78 85.45 6.15 8.73 78.99 60.31 5.98 5.49 31.97 0.92 2.44 12.81 18.02 13.68 35.59 4.79 22.15 6.4 3.25 1.52 7.13 2.09 18.11 30.01 25.32 3.72 26.04 34.96 3.17 13.11 6.14 8.34 10.01 62.99 100.38 3.01 6.43 353.58 5.27 4.95 106.01 92.15 20.71 12.04 4.5 42.33 92.12 31.58 1.49 6.14 10.85 31.92 21.98 83.06 38.52 347.51 24.66 6.09 11.53 425.75 32.41 608.74 16.3 49.39 0.78 36.59 302.44 15.87 20.68 140.87 96.08 

We can now download more results files from other bundles by re-using our query results:

In [99]:
the_second_bundle = bundles['results'][1]
bundle_files = the_second_bundle['metadata']['manifest']['files']
for f in bundle_files:
    if f['name'].endswith('.results'):
        results_file_uuid, results_file_version = f['uuid'], f['version']

With the new results file UUID, we can get the `.results` file itself using a `get_file()` API call:

In [104]:
results2 = client.get_file(replica='aws', uuid=results_file_uuid, version=results_file_version)
open('matrix2.results', 'w').write(results2.decode("utf-8"))

18886773

As before, we extract data from the results file, filter it, and write it to a new file using a csv writer and a csv reader:

In [106]:
with open('matrix2.results', 'r') as infile, open('matrix2.tsv', 'w', newline='') as outfile:
    reader = csv.DictReader(infile, delimiter='\t')
    writer = csv.DictWriter(outfile, fieldnames=['gene_id', 'TPM', 'FPKM'], delimiter='\t')
    for row in reader:
        writer.writerow({'gene_id': row['gene_id'], 'TPM': row['TPM'], 'FPKM': row['FPKM']})

Now create a new Scanpy AnnData object:

In [107]:
adata2 = sc.read_csv( filename='matrix.tsv', delimiter='\t' ).transpose()
print(adata2)

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


AnnData object with n_obs × n_vars = 2 × 200468 


As before, we can access the data using the AnnData object's built-in methods or by using the `adata2.X` attribute:

In [108]:
for i in range(0, 153):
    print( '{:<6}'.format('{:.1f}'.format(adata2.X[0][i])), end='' if (i + 1) % 17 != 0 else '\n' )

92.3  0.0   0.0   4.1   0.0   0.0   0.0   0.0   0.0   194.4 0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0.0   0.0   43.9  102.7 0.0   3.2   0.0   35.6  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0.0   0.0   0.0   0.0   0.0   0.0   78.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0.0   0.0   0.0   0.0   0.0   20.1  69.3  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0.0   18.4  20.9  53.4  0.0   0.0   0.0   0.0   135.5 0.0   0.0   145.7 0.0   0.0   0.0   0.0   0.0   
0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   7.4   0.0   0.0   0.0   0.0   0.0   39.8  


Now we have two matrices in Scanpy, aand we are ready to perform our analysis of choice in Scanpy. This concludes the download tutorial.